# Importaciones para el proyecto:

In [ ]:
import sys
import os
# os.getcwd()
sys.path.append(os.path.abspath(".."))

'c:\\Users\\Usuario\\REPO_BOOTCAMP\\ML_Adidas-Forecast\\src\\results_notebook'

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import utils.bootcampviztools as bt
import utils.funciones_eda as fe

# ✅ Step 1: Objetivo del proyecto.


### ADIDAS Y SUS DISTRIBUIDORES EN UNITED STATES. 🔎

Adidas quiere saber una predicción razonable de cuánto facturarán las tiendas distribuidoras para asi saber si los productos que solicitan son adecuados a su nivel de facturación. Es muy importante que el producto esté en el lugar más eficiente posible para ello debemos predecirlo.

A su vez, quieren plantear un sistema logístico para los Estados de USA donde haya más demanda de producto. Considerando asi una prevención a la rotura de stock y a la mejora en costes ya que el producto más vendido en cada Estado estaría más cerca de dichas tiendas distribuidoras.

🤔 PREGUNTAS QUE NOS HACEMOS 🤔

A. PREDECIR LA FACTURACIÓN DE LAS TIENDAS DISTRIBUIDORAS 💵
- ¿Qué mercado es el que está en auge para estos años?
- ¿Hay algún mercado que debamos activarnos porque va en declive?
- ¿Cumplirán las tiendas con el mínimo de facturación deseable?
- ¿Varía la facturación con la estacionalidad?

B. PREVENCIÓN A LA ROTURA DE STOCK Y MEJORA EN COSTES 🚛
- ¿En qué regiones debemos tener un centro logístico más potente para el futuro?
- ¿Cuánto stock por categoría deberá estar disponible en cada región?

# ✅ Step 2: Obtener datos.


In [80]:
file_path = "../data/adidas_sales.csv"

In [81]:
df = pd.read_csv(file_path)
df.head(5)

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Sales Method
0,Walmart,1128299,6/17/2021,Southeast,Florida,Orlando,Women's Apparel,$103.00,218,"2,245","$1,257",Online
1,West Gear,1128299,7/16/2021,South,Louisiana,New Orleans,Women's Apparel,$103.00,163,"1,679",$806,Online
2,Sports Direct,1197831,8/25/2021,South,Alabama,Birmingham,Men's Street Footwear,$10.00,700,"7,000","$3,150",Outlet
3,Sports Direct,1197831,8/27/2021,South,Alabama,Birmingham,Women's Street Footwear,$15.00,575,"8,625","$3,881",Outlet
4,Sports Direct,1197831,8/21/2021,South,Alabama,Birmingham,Women's Street Footwear,$15.00,475,"7,125","$3,206",Outlet


---------
A simple vista vemos:
- Columnas espaciadas.
- Error en el cálculo de ***"Total Sales"***
- Columnas que vamos a limpiar para tener el DataFrame lo más funcional posible.

Y más cosas que veremos en el siguiente paso

# ✅ Step 3: Limpieza de datos - Corecciones.


- Solucionamos las columnas espaciadas:

In [82]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9641 entries, 0 to 9640
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   retailer          9641 non-null   object
 1   retailer_id       9641 non-null   int64 
 2   invoice_date      9641 non-null   object
 3   region            9641 non-null   object
 4   state             9641 non-null   object
 5   city              9641 non-null   object
 6   product           9641 non-null   object
 7   price_per_unit    9639 non-null   object
 8   units_sold        9641 non-null   object
 9   total_sales       9641 non-null   object
 10  operating_profit  9641 non-null   object
 11  sales_method      9641 non-null   object
dtypes: int64(1), object(11)
memory usage: 904.0+ KB


- Valores faltantes en ***"price_per_unit"*** pero antes solucionaremos el error de cálculo en ***"total_sales"***
- Tenemos tipo object en todas las columnas, debemos transformar a númericas algunas.
- Queremos transformar ***"invoice_date"*** en datetime.
- Vamos a eliminar columnas no relevantes al proyecto.

In [84]:
df.sample(1)

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
1885,Kohl's,1185732,9/6/2021,Northeast,New Jersey,Newark,Women's Street Footwear,$40.00,175,"7,000","$2,800",In-store


- Si vemos el *sample* podemos ver que la multiplicación en ***"total_sales"*** (*price_per_unit* y *units_sold*) no es correcta. Vamos a hacerla de nuevo y corregiremos el símbolo del dollar. La haremos tipo numérica. Por último añadiremos dos decimales en las columnas.

In [85]:
df["price_per_unit"] = df["price_per_unit"].str.replace("$", "")
df["price_per_unit"] = pd.to_numeric(df["price_per_unit"], errors="coerce")

In [86]:
# Visión general de los datos
df.sample(6)

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
8323,Foot Locker,1185732,10/22/2021,South,Texas,Dallas,Men's Street Footwear,45.0,425,"19,125","$10,519",Outlet
6604,West Gear,1128299,1/6/2021,West,California,San Francisco,Men's Street Footwear,60.0,975,"58,500","$23,400",Outlet
3897,West Gear,1128299,7/18/2020,West,Washington,Seattle,Women's Apparel,62.0,113,701,$245,Online
6152,Sports Direct,1185732,8/9/2021,Midwest,North Dakota,Fargo,Men's Apparel,46.0,44,202,$93,Online
7564,Foot Locker,1128299,9/24/2021,West,Arizona,Phoenix,Men's Street Footwear,45.0,575,"25,875","$9,056",Online
8473,West Gear,1128299,1/28/2020,West,Nevada,Las Vegas,Women's Street Footwear,65.0,400,"26,000","$11,700",Outlet


In [87]:
df["units_sold"] = pd.to_numeric(df["units_sold"].str.replace(",", ""),errors="coerce")


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9641 entries, 0 to 9640
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   retailer          9641 non-null   object 
 1   retailer_id       9641 non-null   int64  
 2   invoice_date      9641 non-null   object 
 3   region            9641 non-null   object 
 4   state             9641 non-null   object 
 5   city              9641 non-null   object 
 6   product           9641 non-null   object 
 7   price_per_unit    9639 non-null   float64
 8   units_sold        9641 non-null   int64  
 9   total_sales       9641 non-null   object 
 10  operating_profit  9641 non-null   object 
 11  sales_method      9641 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 904.0+ KB


Siendo columnas numéricas ya podríamos calcular el correcto "total_sales" pero vamos a visualizar los nulos primero antes de tocar los datos.

In [89]:
df[df.isnull().any(axis=1)]

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
6725,Foot Locker,1185732,1/29/2020,Northeast,New York,New York,Men's Apparel,NaN,850,"51,000","$15,300",Outlet
7882,Sports Direct,1197831,7/29/2020,South,Texas,Houston,Men's Apparel,NaN,500,"20,000","$6,000",Outlet


---------
🤔 Esto seria una pregunta para "Negocio" porque tenemos dos soluciones para afrontar estos nulos.
1. Entendemos que la división de ***"total_sales"*** / ***"units_sold"*** dan el resultado de ***"price_per_unit"***
2. Hemos visto que hay valores de ***"total_sales"*** que no fueron bien calculados y todos tienen en común la falta del último dígito.

Ya que son dos valores faltantes, vamos a confiar en los valores que tenemos del dataset y haremos la opción 1

In [90]:
df["total_sales"] = pd.to_numeric(df["total_sales"].str.replace(",", ""),errors="coerce")


In [91]:
df["price_per_unit"] = df["price_per_unit"].fillna(df["total_sales"] / df["units_sold"])

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9641 entries, 0 to 9640
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   retailer          9641 non-null   object 
 1   retailer_id       9641 non-null   int64  
 2   invoice_date      9641 non-null   object 
 3   region            9641 non-null   object 
 4   state             9641 non-null   object 
 5   city              9641 non-null   object 
 6   product           9641 non-null   object 
 7   price_per_unit    9641 non-null   float64
 8   units_sold        9641 non-null   int64  
 9   total_sales       9641 non-null   int64  
 10  operating_profit  9641 non-null   object 
 11  sales_method      9641 non-null   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 904.0+ KB


- Calculamos los valores correctos de ***"total_sales"***

In [93]:
df['total_sales'] = df['price_per_unit'] * df['units_sold']

In [94]:
df.sample(5)

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
8911,Foot Locker,1128299,4/10/2021,West,Hawaii,Honolulu,Women's Apparel,70.0,325,22750.0,"$9,100",Outlet
6779,Sports Direct,1197831,4/15/2021,South,Tennessee,Knoxville,Men's Street Footwear,40.0,825,33000.0,"$14,850",Outlet
9507,Sports Direct,1185732,6/15/2021,Northeast,Massachusetts,Boston,Women's Athletic Footwear,45.0,225,10125.0,"$2,531",In-store
1617,West Gear,1128299,5/21/2020,West,Colorado,Denver,Men's Athletic Footwear,64.0,182,11648.0,$443,Online
8169,Foot Locker,1185732,10/19/2021,Northeast,New Hampshire,Manchester,Men's Street Footwear,70.0,475,33250.0,"$13,300",Outlet


- DateTime para ***"invoice_date"*** y ordenamos el DataFrame

In [95]:
df["invoice_date"] = pd.to_datetime(df["invoice_date"])

In [101]:
df = df.sort_values(by="invoice_date")
df.head(3)

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
4908,Foot Locker,1185732,2020-01-01,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68.0,83,5644.0,$243,Online
30,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,34.0,384,13056.0,$679,Outlet
4937,Foot Locker,1185732,2020-01-01,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53.0,83,4399.0,$141,Outlet


In [102]:
df.isnull().sum()

retailer            0
retailer_id         0
invoice_date        0
region              0
state               0
city                0
product             0
price_per_unit      0
units_sold          0
total_sales         0
operating_profit    0
sales_method        0
dtype: int64

- Eliminamos columnas irrelevantes y hacemos una copia de seguridad del dataset

In [103]:
df = df.drop(columns=["retailer_id", "operating_profit", "sales_method"])

# "operation_profit" se elimina porque no tenemos contexto sobre los gastos para entender realmente el profit

- Una vista rápida para saber si está todo en orden

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9641 entries, 4908 to 3413
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   retailer        9641 non-null   object        
 1   invoice_date    9641 non-null   datetime64[ns]
 2   region          9641 non-null   object        
 3   state           9641 non-null   object        
 4   city            9641 non-null   object        
 5   product         9641 non-null   object        
 6   price_per_unit  9641 non-null   float64       
 7   units_sold      9641 non-null   int64         
 8   total_sales     9641 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 753.2+ KB


In [105]:
df.region.value_counts()

region
West         2446
Northeast    2376
Midwest      1869
South        1726
Southeast    1224
Name: count, dtype: int64

In [108]:
df.state.value_counts()

state
Texas             430
California        430
New York          360
Florida           360
Mississippi       216
Massachusetts     216
Oklahoma          216
Rhode Island      216
Oregon            216
Virginia          216
Vermont           216
Pennsylvania      216
Idaho             216
New Mexico        216
Utah              216
New Hampshire     216
Arizona           216
Tennessee         216
Arkansas          216
Alabama           216
Nevada            216
Connecticut       216
Georgia           216
Louisiana         216
Delaware          144
South Carolina    144
Nebraska          144
Colorado          144
West Virginia     144
Wyoming           144
Washington        144
Minnesota         144
Montana           144
Iowa              144
Ohio              144
Kentucky          144
Alaska            144
North Dakota      144
Michigan          144
Maryland          144
Hawaii            144
New Jersey        144
Indiana           144
South Dakota      144
Wisconsin         144
Main

In [109]:
df.city.value_counts()

city
Portland          360
Charleston        288
Orlando           216
Albuquerque       216
New York          216
Boston            216
Oklahoma City     216
Providence        216
Richmond          216
Burlington        216
Philadelphia      216
Boise             216
Little Rock       216
Salt Lake City    216
Manchester        216
Phoenix           216
Los Angeles       216
Knoxville         216
Birmingham        216
Jackson           216
Dallas            216
New Orleans       216
Las Vegas         216
Hartford          216
Atlanta           216
Houston           214
San Francisco     214
Anchorage         144
Omaha             144
Denver            144
Miami             144
Seattle           144
Cheyenne          144
Minneapolis       144
Billings          144
Des Moines        144
Columbus          144
Louisville        144
St. Louis         144
Fargo             144
Detroit           144
Baltimore         144
Honolulu          144
Indianapolis      144
Newark            144
Sioux

In [112]:
df["product"].value_counts()

product
Men's Street Footwear        1610
Men's Athletic Footwear      1608
Women's Apparel              1607
Women's Street Footwear      1606
Women's Athletic Footwear    1605
Men's Apparel                1604
Men's aparel                    1
Name: count, dtype: int64

In [114]:
# Error de ortografía
df["product"] = df["product"].replace({"Men's aparel": "Men's Apparel"})
df["product"].value_counts()

product
Men's Street Footwear        1610
Men's Athletic Footwear      1608
Women's Apparel              1607
Women's Street Footwear      1606
Women's Athletic Footwear    1605
Men's Apparel                1605
Name: count, dtype: int64

In [115]:
df.retailer.value_counts()

retailer
Foot Locker      2634
West Gear        2372
Sports Direct    2030
Kohl's           1030
Amazon            949
Walmart           626
Name: count, dtype: int64

In [116]:
df.to_csv("../data/adidas_sales_cleaned.csv", index=False)

# ✅ Step 4: Train y Test.


# ✅ Step 5: Mini Eda.


# ✅ Step 6: Preparación del dataset de Train.


# ✅ Step 7: Selección e instanciación de modelos. Baseline.


# ✅ Step 8: Comparación de modelos.


# ✅ Step 9: Selección del modelo - Optimización de hiperparámetros.


# ✅ Step 10: Evaluación contra test.


# ✅ Step 11: Análisis de errores.


# ✅ Step 12: Persistencia del modelo en disco.